In [1]:
import pandas as pd

# Load the datasets
vol_surface_df = pd.read_csv('/Users/jorambakker/Downloads/Vol-surface 30and60.csv')
sp500_df = pd.read_csv('/Users/jorambakker/Downloads/SP500.csv')

# Convert the 'date' columns to datetime if not already done
vol_surface_df['date'] = pd.to_datetime(vol_surface_df['date'])#, format = '%Y-%M-%d')
sp500_df['date'] = pd.to_datetime(sp500_df['date'])#,format = '%Y-%M-%d')




In [2]:
##This is for computing the correct file for Vilkov python code

# Merge the datasets on the 'date' column
combined_df = pd.merge(vol_surface_df, sp500_df, on='date', how='left')

#calculate moneyness
combined_df['mnes'] = combined_df.apply(lambda row: row['close'] / row['impl_strike'],  axis=1)

# Save the merged dataset
combined_df.to_csv('Thesis data.csv', index=False)

In [3]:
combined_df

,secid_x,date,days,delta,impl_volatility,impl_strike,impl_premium,dispersion,cp_flag,sic,index_flag,secid_y,open,close,mnes
0,108105,1996-01-04,30,-90,0.138704,652.5124,34.12913,0.007301,P,9999,1,108105,621.32,617.70,0.946649
1,108105,1996-01-04,30,-85,0.137368,645.8368,28.19517,0.007203,P,9999,1,108105,621.32,617.70,0.956434
2,108105,1996-01-04,30,-80,0.135625,640.5763,23.73218,0.007088,P,9999,1,108105,621.32,617.70,0.964288
3,108105,1996-01-04,30,-75,0.132932,636.0494,20.03727,0.007155,P,9999,1,108105,621.32,617.70,0.971151
4,108105,1996-01-04,30,-70,0.128539,631.9539,16.76515,0.006979,P,9999,1,108105,621.32,617.70,0.977445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464775,108105,2023-02-28,60,70,0.223737,3818.3980,241.29800,0.014961,C,9999,1,108105,3977.19,3970.15,1.039742
464776,108105,2023-02-28,60,75,0.234951,3756.4640,290.05430,0.023737,C,9999,1,108105,3977.19,3970.15,1.056885
464777,108105,2023-02-28,60,80,0.251865,3677.7920,356.50980,0.044793,C,9999,1,108105,3977.19,3970.15,1.079493
464778,108105,2023-02-28,60,85,0.278868,3567.9120,454.77150,0.080997,C,9999,1,108105,3977.19,3970.15,1.112738


In [4]:
vilkov_df = pd.DataFrame()
vilkov_df["id"] = combined_df["secid_x"]
vilkov_df["date"] = combined_df["date"]
vilkov_df["days"] = combined_df["days"]
vilkov_df["impl_volatility"] = combined_df["impl_volatility"]
vilkov_df["mnes"] = combined_df["mnes"]
vilkov_df["prem"] = combined_df["impl_premium"]/combined_df["close"]
vilkov_df["delta"] = combined_df["delta"]

vilkov_df_30 = vilkov_df[vilkov_df["days"]==30]
vilkov_df_60 = vilkov_df[vilkov_df["days"]==60]

vilkov_df

vilkov_df_30.to_csv('Vilkov30.csv', index=False)
vilkov_df_60.to_csv('Vilkov60.csv', index=False)
            

In [5]:
## If not using Vilkov, then I should take the average of the most ATM put and the call volatility

# Filter for days till maturity being either 30 or 60 days
filtered_data = combined_df[combined_df['days'].isin([30, 60])]

# Function to find the row with moneyness closest to 1 for each group
def closest_moneyness(group):
    return group.iloc[(group['mnes'] - 1).abs().argmin()]

# Group the data by 'date', 'cp_flag', and 'days', then apply the function
grouped_data = filtered_data.groupby(['date', 'cp_flag', 'days']).apply(closest_moneyness).reset_index(drop=True)

# Save the filtered and grouped dataset to a new CSV file
grouped_data.to_csv('Filtered_Thesis_Data.csv', index=False)



# Now, I transform the volatility to variance
grouped_data['impl_variance'] = grouped_data['impl_volatility'] ** 2
#print(grouped_data)

# Now, group by 'date' and 'days' only, calculate average implied variance
average_variance = grouped_data.groupby(['date', 'days'])['impl_variance'].mean().reset_index()
#print(average_variance)

# Pivot the DataFrame to create separate columns for each maturity
pivot_avg_variance = average_variance.pivot(index='date', columns='days', values='impl_variance')
pivot_avg_variance.columns = [f'avg_impl_variance{col}' for col in pivot_avg_variance.columns]

# Reset index to convert 'date' from index to a column
final_data = pivot_avg_variance.reset_index(drop=False)

# Save the final DataFrame to a CSV file
# This is daily data from the avg_implied_variance of 30 and 60 days.
final_data.to_csv('output_avg_variance.csv', index=False)


/var/folders/wn/lk75hc7s65b7ljzsdgw30f7c0000gn/T/ipykernel_17831/4074053363.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_data = filtered_data.groupby(['date', 'cp_flag', 'days']).apply(closest_moneyness).reset_index(drop=True)


In [6]:
final_data

,date,avg_impl_variance30,avg_impl_variance60
0,1996-01-04,0.012734,0.013009
1,1996-01-05,0.012810,0.013393
2,1996-01-08,0.012789,0.013384
3,1996-01-09,0.017450,0.016158
4,1996-01-10,0.018594,0.018308
...,...,...,...
6830,2023-02-22,0.041411,0.039590
6831,2023-02-23,0.037493,0.036334
6832,2023-02-24,0.039970,0.038902
6833,2023-02-27,0.038791,0.037165


In [7]:
##Vilkov MFIV_BJN
vk30 = pd.read_csv('/Users/jorambakker/Downloads/OICodeOSF/Vilkov30_out.csv')
vk30['date'] = pd.to_datetime(vk30['date'])
vk60 = pd.read_csv('/Users/jorambakker/Downloads/OICodeOSF/Vilkov60_out.csv')
vk60['date'] = pd.to_datetime(vk60['date'])

vk3060 = pd.DataFrame()
vk3060["date"] = vk30["date"]
vk3060["avg_impl_variance30"] = vk30["mfiv_bjn"]
vk3060["avg_impl_variance60"] = vk60["mfiv_bjn"]

In [8]:
#vk3060[1070:1120]

In [9]:
## Convert to only first trading day of month

#df = final_data
df = vk3060
#print(df)
df.sort_values('date', inplace=True)

# Group by year and month, then select the first row of each group
#first_of_month = df.groupby([df['date'].dt.year, df['date'].dt.month]).first().reset_index(drop=True)
#last
#first_of_month = df.groupby([df['date'].dt.year, df['date'].dt.month]).last().reset_index(drop=True)



In [10]:
# Create a mask that is True for rows where the 'date' is the third Friday of its month
mask = df['date'].apply(lambda x: 
    (x.weekday() == 4) and            # It's a Friday
    (15 <= x.day <= 21)               # It's between the 15th and the 21st
)

# Apply the mask to the DataFrame
third_fridays_df = df[mask]

# Reset index if necessary
third_fridays_df = third_fridays_df.reset_index(drop=True)

In [11]:
third_fridays_df

,date,avg_impl_variance30,avg_impl_variance60
0,1996-01-19,0.016204,0.017181
1,1996-02-16,0.023784,0.023006
2,1996-03-15,0.026492,0.025748
3,1996-04-19,0.022736,0.025563
4,1996-05-17,0.022694,0.022965
...,...,...,...
315,2022-10-21,0.082994,0.080109
316,2022-11-18,0.048704,0.052136
317,2022-12-16,0.044115,0.051242
318,2023-01-20,0.036207,0.039653


In [12]:
#first_of_month = third_fridays_df

# To calculate the forecast at t-1 of the period between t,t+h
first_of_month = df
first_of_month['Forecast,t-1 of X(t,t+h)'] = (first_of_month['avg_impl_variance60']*60/30 - first_of_month['avg_impl_variance30']*30/30).shift(21)

first_of_month.to_csv('CG data thesis.csv', index=False)
first_of_month

,date,avg_impl_variance30,avg_impl_variance60,"Forecast,t-1 of X(t,t+h)"
0,1996-01-04,0.017401,0.017632,NaN
1,1996-01-05,0.016994,0.017460,NaN
2,1996-01-08,0.016003,0.016644,NaN
3,1996-01-09,0.023240,0.019927,NaN
4,1996-01-10,0.023445,0.021856,NaN
...,...,...,...,...
6830,2023-02-22,0.044359,0.043985,0.042902
6831,2023-02-23,0.039787,0.040185,0.041665
6832,2023-02-24,0.042340,0.042786,0.041284
6833,2023-02-27,0.040849,0.040473,0.040709


In [13]:
## For calculating the annualized variance by myself

df = sp500_df

import numpy as np
def calculate_annualized_variance(df):
    # Set index to date
    df.set_index('date', inplace=True)
    
    # Calculate daily log returns
    df['Log Returns'] = np.log(df['close'] / df['close'].shift(1))

    # Initialize the column for realized variance
    df['Realized Variance'] = np.nan
    df['Annualized Variance'] = np.nan

    # Iterate over the DataFrame to set dynamic windows based on dates
    for index in df.index:
        # Find the date 30 calendar days ahead
        target_date = index + pd.Timedelta(days=30)

        # Filter the data to get returns from the current date to the target date
        window_data = df['Log Returns'][(df.index > index) & (df.index <= target_date)]

        # Calculate the variance of log returns in the window and assign it to the current date
        if not window_data.empty:
            realized_variance = window_data.var()
            df.at[index, 'Realized Variance'] = realized_variance
            # Annualize the variance
            df.at[index, 'Annualized Variance'] = realized_variance * (252)

    return df

new_df = calculate_annualized_variance(df)
new_df = new_df.reset_index(drop=False)
new_df

,date,secid,open,close,Log Returns,Realized Variance,Annualized Variance
0,1996-01-04,108105,621.32,617.70,NaN,0.000064,0.016155
1,1996-01-05,108105,617.70,616.71,-0.001604,0.000067,0.016882
2,1996-01-08,108105,616.71,618.46,0.002834,0.000065,0.016379
3,1996-01-09,108105,618.46,609.45,-0.014676,0.000053,0.013235
4,1996-01-10,108105,609.45,598.48,-0.018164,0.000030,0.007592
...,...,...,...,...,...,...,...
6830,2023-02-22,108105,4001.83,3991.05,-0.001575,0.000051,0.012788
6831,2023-02-23,108105,4018.60,4012.32,0.005315,0.000047,0.011802
6832,2023-02-24,108105,3973.24,3970.04,-0.010593,0.000019,0.004702
6833,2023-02-27,108105,3992.36,3982.24,0.003068,NaN,NaN


In [14]:
#Combining self calculated variance to the first of month data (with the avg_implieds)
cg = pd.merge(first_of_month, new_df[["date", "Annualized Variance"]], on='date', how='left')

cg.to_csv('CG data thesis.csv', index=False)
cg

,date,avg_impl_variance30,avg_impl_variance60,"Forecast,t-1 of X(t,t+h)",Annualized Variance
0,1996-01-04,0.017401,0.017632,NaN,0.016155
1,1996-01-05,0.016994,0.017460,NaN,0.016882
2,1996-01-08,0.016003,0.016644,NaN,0.016379
3,1996-01-09,0.023240,0.019927,NaN,0.013235
4,1996-01-10,0.023445,0.021856,NaN,0.007592
...,...,...,...,...,...
6830,2023-02-22,0.044359,0.043985,0.042902,0.012788
6831,2023-02-23,0.039787,0.040185,0.041665,0.011802
6832,2023-02-24,0.042340,0.042786,0.041284,0.004702
6833,2023-02-27,0.040849,0.040473,0.040709,NaN


In [15]:
#Historical Vol based on OptionMetrics
hisvol = pd.read_csv('/Users/jorambakker/Downloads/HisVol.csv')
hisvol['date'] = pd.to_datetime(hisvol['date'])
cg1 = pd.merge(cg, hisvol[['date','volatility']], on='date', how='left')

cg1['historical variance'] = (cg1['volatility']**2).shift(-21)


In [16]:
hisvol

,secid,date,days,volatility,cusip,index_flag
0,108105,1996-01-04,30,0.099637,64881510,1
1,108105,1996-01-05,30,0.098612,64881510,1
2,108105,1996-01-08,30,0.098716,64881510,1
3,108105,1996-01-09,30,0.109438,64881510,1
4,108105,1996-01-10,30,0.124620,64881510,1
...,...,...,...,...,...,...
6830,108105,2023-02-22,30,0.162453,64881510,1
6831,108105,2023-02-23,30,0.163618,64881510,1
6832,108105,2023-02-24,30,0.167645,64881510,1
6833,108105,2023-02-27,30,0.166740,64881510,1


In [17]:
#This is for the Uncertainty parameter that I will add to the CG Model
uncertainty_df= pd.read_excel('/Users/jorambakker/Downloads/MacroFinanceUncertainty_202402Update/RealUncertaintyToCirculate.xlsx')
uncertainty_df['Date'] = pd.to_datetime(uncertainty_df['Date'])

# I need to merge the datasets based on the month
cg1['month_year'] = cg1['date'].dt.to_period('M')
uncertainty_df['month_year'] = uncertainty_df['Date'].dt.to_period('M')

cg_unc = pd.merge(cg1, uncertainty_df, on='month_year', how='left')
cg_unc["lag_h1"] = cg_unc["h=1"].shift(21)

cg_unc['h1_change'] = np.log(cg_unc['h=1'] / cg_unc['h=1'].shift(21))
cg_unc.to_csv('CG Uncertainty.csv',index=False)

In [18]:

cg_unc

,date,avg_impl_variance30,avg_impl_variance60,"Forecast,t-1 of X(t,t+h)",Annualized Variance,volatility,historical variance,month_year,Date,h=1,h=3,h=12,lag_h1,h1_change
0,1996-01-04,0.017401,0.017632,NaN,0.016155,0.099637,0.015984,1996-01,1996-01-01,0.626934,0.736389,0.862091,NaN,NaN
1,1996-01-05,0.016994,0.017460,NaN,0.016882,0.098612,0.016664,1996-01,1996-01-01,0.626934,0.736389,0.862091,NaN,NaN
2,1996-01-08,0.016003,0.016644,NaN,0.016379,0.098716,0.016248,1996-01,1996-01-01,0.626934,0.736389,0.862091,NaN,NaN
3,1996-01-09,0.023240,0.019927,NaN,0.013235,0.109438,0.016379,1996-01,1996-01-01,0.626934,0.736389,0.862091,NaN,NaN
4,1996-01-10,0.023445,0.021856,NaN,0.007592,0.124620,0.013235,1996-01,1996-01-01,0.626934,0.736389,0.862091,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6830,2023-02-22,0.044359,0.043985,0.042902,0.012788,0.162453,NaN,2023-02,2023-02-01,0.731666,0.840335,0.942668,0.760922,-0.039206
6831,2023-02-23,0.039787,0.040185,0.041665,0.011802,0.163618,NaN,2023-02,2023-02-01,0.731666,0.840335,0.942668,0.760922,-0.039206
6832,2023-02-24,0.042340,0.042786,0.041284,0.004702,0.167645,NaN,2023-02,2023-02-01,0.731666,0.840335,0.942668,0.760922,-0.039206
6833,2023-02-27,0.040849,0.040473,0.040709,NaN,0.166740,NaN,2023-02,2023-02-01,0.731666,0.840335,0.942668,0.760922,-0.039206


In [19]:
# For Stein method AR1

df = pd.merge(vk3060, hisvol[['date','volatility']],on='date',how='left')
df['historical variance']= (df['volatility']**2)

df['var30_lag1w'] = df['avg_impl_variance30'].shift(5)
df['var30_lag2w'] = df['avg_impl_variance30'].shift(10)
df['var30_lag3w'] = df['avg_impl_variance30'].shift(15)
df['var30_lag4w'] = df['avg_impl_variance30'].shift(20)

df.to_csv('Stein data thesis.csv', index=False)

df[0:30]

,date,avg_impl_variance30,avg_impl_variance60,"Forecast,t-1 of X(t,t+h)",volatility,historical variance,var30_lag1w,var30_lag2w,var30_lag3w,var30_lag4w
0,1996-01-04,0.017401,0.017632,NaN,0.099637,0.009928,NaN,NaN,NaN,NaN
1,1996-01-05,0.016994,0.017460,NaN,0.098612,0.009724,NaN,NaN,NaN,NaN
2,1996-01-08,0.016003,0.016644,NaN,0.098716,0.009745,NaN,NaN,NaN,NaN
3,1996-01-09,0.023240,0.019927,NaN,0.109438,0.011977,NaN,NaN,NaN,NaN
4,1996-01-10,0.023445,0.021856,NaN,0.124620,0.015530,NaN,NaN,NaN,NaN
5,1996-01-11,0.019709,0.019423,NaN,0.128430,0.016494,0.017401,NaN,NaN,NaN
6,1996-01-12,0.019075,0.019247,NaN,0.126454,0.015991,0.016994,NaN,NaN,NaN
7,1996-01-15,0.021598,0.020295,NaN,0.128004,0.016385,0.016003,NaN,NaN,NaN
8,1996-01-16,0.019093,0.018698,NaN,0.136484,0.018628,0.023240,NaN,NaN,NaN
9,1996-01-17,0.019547,0.018827,NaN,0.125185,0.015671,0.023445,NaN,NaN,NaN
